In [2]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
os.environ["http_proxy"] = "http://devproxy.bloomberg.com:82"
os.environ["https_proxy"] = "http://devproxy.bloomberg.com:82"

In [28]:
from transformers import BridgeTowerProcessor, BridgeTowerModel
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
text = "hello world"

processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-base")
model = BridgeTowerModel.from_pretrained("BridgeTower/bridgetower-base")



In [29]:
# Prepare inputs
encoding = processor(
    images=image,
    text=['a cat', 'a dog'],
    return_tensors="pt")
# Forward pass
outputs = model(**encoding)
outputs.keys()

odict_keys(['text_features', 'image_features', 'pooler_output'])

In [30]:
outputs.text_features.shape

torch.Size([2, 4, 768])

In [31]:
outputs.pooler_output.shape

torch.Size([2, 1536])

In [1]:
# bridge tower
from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
import requests
from PIL import Image

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
texts = ["An image of two cats", "an image of one cat"]

processor = BridgeTowerProcessor.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc")
model = BridgeTowerForContrastiveLearning.from_pretrained("BridgeTower/bridgetower-large-itm-mlm-itc")

# # forward pass
# scores = dict()
# for idx, image in enumerate([image, image]):
#     # prepare inputs
#     encoding = processor(
#         image,
#         texts[0],
#         return_tensors="pt", padding=True)
#     outputs = model(**encoding)
#     scores[idx] = outputs

/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BridgeTowerForContrastiveLearning were not initialized from the model checkpoint at BridgeTower/bridgetower-large-itm-mlm-itc and are newly initialized: ['logit_scale']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
k = 10
encoding = processor(
   images=[image]*k,
    text=["An image of two cats"]*k,
    return_tensors="pt", padding=True
)
outputs = model(**encoding)

In [42]:
outputs['text_embeds'].shape

torch.Size([10, 512])

In [ ]:
from src.utils.utils import get_config

config = get_config(dataset='f30k', model='')

In [7]:
# ALIGN
import requests
import torch
from PIL import Image
from transformers import AlignProcessor, AlignModel
from sentence_transformers import SentenceTransformer, util

processor = AlignProcessor.from_pretrained("kakaobrain/align-base")
model = AlignModel.from_pretrained("kakaobrain/align-base")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)
candidate_labels = ["test"]

inputs = processor(text=candidate_labels, images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

# this is the image-text similarity score
logits_per_image = outputs.logits_per_image

# we can take the softmax to get the label probabilities
probs = logits_per_image.softmax(dim=1)
print(probs)


print("outputs['text_embeds'].shape: ", outputs['text_embeds'].shape)
print("outputs['image_embeds'].shape: ", outputs['image_embeds'].shape)

align_text_embs = outputs['text_embeds']
align_img_embs = outputs['image_embeds']

cos_sim_scores = util.cos_sim(align_img_embs, align_text_embs)

cos_sim_scores

tensor([[1.]])
outputs['text_embeds'].shape:  torch.Size([1, 640])
outputs['image_embeds'].shape:  torch.Size([1, 640])


tensor([[0.0846]])

In [3]:
# groupVIT
from PIL import Image
import requests
from transformers import AutoProcessor, GroupViTModel

model = GroupViTModel.from_pretrained("nvidia/groupvit-gcc-yfcc")
processor = AutoProcessor.from_pretrained("nvidia/groupvit-gcc-yfcc")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(
    text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True
)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [22]:
from sentence_transformers import SentenceTransformer, util

inputs = processor(
    text=["a photo of a dog", "a photo of a cat"],
    images=image,
    return_tensors="pt", padding=True
)

outputs = model(**inputs)

txt_emb = outputs['text_embeds']

img_emb = outputs['image_embeds']


util.cos_sim(txt_emb, img_emb)

tensor([[0.0958],
        [0.2010]], grad_fn=<MmBackward0>)

In [8]:
from transformers import CLIPTokenizer, GroupViTModel

model = GroupViTModel.from_pretrained("nvidia/groupvit-gcc-yfcc")
tokenizer = CLIPTokenizer.from_pretrained("nvidia/groupvit-gcc-yfcc")

inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")
text_features = model.get_text_features(**inputs)

In [10]:
import os

def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

path = '/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/config'
path_gen = absoluteFilePaths(path)

len(list(path_gen))

33

In [37]:
task

't2i'

In [16]:
from PIL import Image
import requests
from sentence_transformers import util
from transformers import AltCLIPModel, AltCLIPProcessor

model = AltCLIPModel.from_pretrained("BAAI/AltCLIP")
processor = AltCLIPProcessor.from_pretrained("BAAI/AltCLIP")


url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=["a photo of a cat", "a photo of a dog", 'london'], images=image, return_tensors="pt", padding=True)

outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1)

out_txt_emb = outputs['text_embeds']
out_img_emb = outputs['image_embeds']

util.cos_sim(out_txt_emb, out_img_emb)

tensor([[0.1672],
        [0.0899],
        [0.1058]], grad_fn=<MmBackward0>)

In [6]:
from src.utils.utils import get_config, get_logger
from munch import Munch
from src.evaluation.evaluator import Evaluator
import argparse

class Args(argparse.Namespace):
  dataset = 'f30k'
  model = 'groupvit'
  task = 't2i'
  compute_from_scratch = True
  perturbation = 'none'

args=Args()

logging = get_logger(args=args, task='i2t')

config = get_config(dataset=args.dataset, model=args.model)

config.args = Munch(
    dataset=args.dataset,
    model=args.model, 
    task=args.task,
    perturbation=args.perturbation,
    compute_from_scratch=args.compute_from_scratch,
)
config.logging = logging

evaluator = Evaluator(config=config)

if args.task == 'i2t':
    results = evaluator.i2t()
elif args.task == 't2i':
    results = evaluator.t2i()
else:
    print('Uknown task type, options: i2t, t2i')
print(results.describe())
print(results.head())

Config path:  ./config/groupvit/f30k/development_local.yaml
Loaded config path
Loaded annotations from  /Users/mhendriksen/Desktop/repositories/datasets/f30k/annotations/dataset_flickr30k.json
Computing image embeddings...
Progress: 0/32


KeyboardInterrupt: 

In [5]:
evaluator = Evaluator(config=config)

if args.task == 'i2t':
    results = evaluator.i2t()
elif args.task == 't2i':
    results = evaluator.t2i()
else:
    print('Uknown task type, options: i2t, t2i')
print(results.describe())
print(results.head())

Loaded annotations from  /Users/mhendriksen/Desktop/repositories/datasets/f30k/annotations/dataset_flickr30k.json
Computing caption embeddings...
Progress: 0/5
Progress: 1/5
Progress: 2/5


KeyboardInterrupt: 

In [14]:
config.logging

<module 'logging' from '/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/logging/__init__.py'>

Loaded annotations from  /Users/mhendriksen/Desktop/repositories/datasets/f30k/annotations/dataset_flickr30k.json


KeyboardInterrupt: 

In [54]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
from transformers import FlavaProcessor, FlavaModel, FlavaFeatureExtractor, BertTokenizer

# Load FLAVA model
model = FlavaModel.from_pretrained("facebook/flava-full")
processor = FlavaProcessor.from_pretrained("facebook/flava-full")
feature_extractor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")

# Encode an image:
image = Image.open('two-dogs.jpeg')
inputs =  feature_extractor(images=[image], return_tensors="pt")
outputs = model(**inputs)
image_embeddings = outputs.image_embeddings
print(image_embeddings.shape)
img_emb_pooled = outputs['image_output']['pooler_output']

# Encode text descriptions
# text_emb = model.encode(['Two dogs in the snow', 'A cat on a table', 'A picture of London at night'])
tokenizer = BertTokenizer.from_pretrained("facebook/flava-full")
inputs = tokenizer(['A dog in the snow', 'A cat on a table', 'A picture of London at night', 'test'], return_tensors="pt", padding="max_length", max_length=77)
outputs = model(**inputs)
text_embeddings = outputs.text_embeddings
txt_emb_pooled = outputs['text_output']['pooler_output']

cos_sim_scores = util.cos_sim(img_emb_pooled, txt_emb_pooled)

cos_sim_scores

`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.
/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/.venv/lib/python3.9/site-packages/transformers/models/flava/feature_extraction_flava.py:28: FutureWarning: The class FlavaFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use FlavaImageProcessor instead.
  warnings.warn(


torch.Size([1, 197, 768])


/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/.venv/lib/python3.9/site-packages/transformers/modeling_utils.py:900: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tensor([[-0.0788, -0.0182, -0.0690, -0.0631]], grad_fn=<MmBackward0>)

In [2]:
from src.utils.utils import get_config
from src.data.dataset import Dataset
from src.utils.dataset_preprocessing import load_json_annotations
from src.models.encoders.flava import FLAVA
from src.models.encoders.clip import CLIP

config = get_config(dataset='f30k', model='flava')

model = FLAVA(config=config)

json_file = load_json_annotations(config=config)

ds_test_split = Dataset(config=config, split="test", json_file=json_file)



/Users/mhendriksen/Desktop/repositories/evaluating-cmr-in-mm/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Config path:  ./config/flava/f30k/development_local.yaml
Loaded config path


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /facebook/flava-full/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x2a3009eb0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 115334f7-7bbf-4356-9f22-a70cd6d6f4c7)')' thrown while requesting HEAD https://huggingface.co/facebook/flava-full/resolve/main/config.json
`text_config_dict` is provided which will be used to initialize `FlavaTextConfig`. The value `text_config["id2label"]` will be overriden.
`multimodal_config_dict` is provided which will be used to initialize `FlavaMultimodalConfig`. The value `multimodal_config["id2label"]` will be overriden.
`image_codebook_config_dict` is provided which will be used to initialize `FlavaImageCodebookConfig`. The value `image_codebook_config["id2label"]` will be overriden.
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retri

In [48]:
data = model.compute_caption_embeddings(ds_split=ds_test_split, compute_from_scratch=True)

Computing caption embeddings...
Progress: 0/1
Saving captions...
Saved files to  /Users/mhendriksen/Desktop/repositories/datasets/f30k/flava/f30k-capt-embeddings.pkl


In [62]:
data = model.compute_image_embeddings(compute_from_scratch=True)

Computing image embeddings...
Progress: 0/1
Saving images...
Saved files to  /Users/mhendriksen/Desktop/repositories/datasets/f30k/flava/f30k-img-embeddings.pkl


In [41]:
# load precomputed image embeddings
capt_ids, capts, capt_embs = model.compute_caption_embeddings(
    ds_split=ds_test_split
)

Computing caption embeddings...
Saving captions...
Saved files to  /Users/mhendriksen/Desktop/repositories/datasets/f30k/flava/f30k-capt-embeddings.pkl


In [ ]:
from src.retrieval.retriever import Retriever
from src.metrics.recall_at_k import recall_at_k
from src.models.relevance_estimators.clip_based import RelevanceEstimator
from src.metrics.dcg import DCG


rel_estimator = RelevanceEstimator(config=config, dataset=ds_test_split)
retriever = Retriever(config=config, model=model)
dcg = DCG(config=config, rel_estimator=rel_estimator)

i2t_queries = []
i2t_targets = []
i2t_retrieved_documents = []
i2t_scores = []
i2t_recalls_at_1 = []
i2t_recalls_at_5 = []
i2t_recalls_at_10 = []
i2t_dcgs = []

print("Image-to-text evaluation...")
seen_image_ids = []

from tqdm import tqdm

for datapoint in tqdm(ds_test_split):
    # get textual query and target
    query = datapoint[1]
    target_filename = datapoint[3]

    if target_filename in seen_image_ids:
        continue
    seen_image_ids.append(target_filename)

    retrieved_caption_ids, scores = retriever.retrieve_top_k(
        query=query, documents=capt_embs, documents_names=capt_ids, k=10
    )

    associated_img_ids = [
        ds_test_split.captions[capt_id]["imgid"]
        for capt_id in retrieved_caption_ids
    ]

In [ ]:
from PIL import Image
import requests

from transformers import FlavaProcessor, FlavaModel

model = FlavaModel.from_pretrained("facebook/flava-full")
processor = FlavaProcessor.from_pretrained("facebook/flava-full")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(
  text=["a photo of a cat", "a photo of a dog"], images=[image, image], return_tensors="pt", padding="max_length", max_length=77
)

outputs = model(**inputs)
image_embeddings = outputs.image_embeddings # Batch size X (Number of image patches + 1) x Hidden size => 2 X 197 X 768
text_embeddings = outputs.text_embeddings # Batch size X (Text sequence length + 1) X Hidden size => 2 X 77 X 768
multimodal_embeddings = outputs.multimodal_embeddings # Batch size X (Number of image patches + Text Sequence Length + 3) X Hidden size => 2 X 275 x 768
# Multimodal embeddings can be used for multimodal tasks such as VQA


## Pass only image
from transformers import FlavaFeatureExtractor

feature_extractor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")
inputs = feature_extractor(images=[image, image], return_tensors="pt")
outputs = model(**inputs)
image_embeddings = outputs.image_embeddings

## Pass only image
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("facebook/flava-full")
inputs = tokenizer(["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding="max_length", max_length=77)
outputs = model(**inputs)
text_embeddings = outputs.text_embeddings

In [ ]:
## Pass only image
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("facebook/flava-full")
inputs = tokenizer(["a photo of a cat", "a photo of a dog"], return_tensors="pt", padding="max_length", max_length=77)
outputs = model(**inputs)

In [ ]:
import numpy as np

np.mean(text_embeddings.detach().numpy(), axis=1).shape

In [ ]:
from PIL import Image
import requests

from transformers import FlavaFeatureExtractor, FlavaModel

model = FlavaModel.from_pretrained("facebook/flava-full")
feature_extractor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = feature_extractor(images=[image], return_tensors="pt")

image_embedding = model.get_image_features(**inputs)

print('Image embedding: ', image_embedding.shape)

In [ ]:
from PIL import Image
import requests

from transformers import FlavaFeatureExtractor, FlavaImageModel

model = FlavaImageModel.from_pretrained("facebook/flava-full")
feature_extractor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = feature_extractor(images=[image], return_tensors="pt")

outputs = model(**inputs)
image_embeddings_lhs = outputs.last_hidden_state
print('image_embeddings_lhs: ', image_embeddings_lhs.shape)

In [ ]:
from PIL import Image

from transformers import BertTokenizer, FlavaTextModel

model = FlavaTextModel.from_pretrained("facebook/flava-full")
tokenizer = BertTokenizer.from_pretrained("facebook/flava-full")

inputs = tokenizer(text=["a photo of a dog"], return_tensors="pt", padding="max_length", max_length=77)

outputs = model(**inputs)
text_embeddings_lhs = outputs.last_hidden_state
print('text_embeddings_lhs: ', text_embeddings_lhs.shape)

In [ ]:
image_embeddings_lhs.shape

In [ ]:
from PIL import Image
import requests

from transformers import FlavaFeatureExtractor, FlavaImageModel

model = FlavaImageModel.from_pretrained("facebook/flava-full")
feature_extractor = FlavaFeatureExtractor.from_pretrained("facebook/flava-full")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

inputs = feature_extractor(images=[image], return_tensors="pt")

outputs = model(**inputs)
image_embeddings_lhs = outputs.last_hidden_state


In [ ]:
from PIL import Image

from transformers import BertTokenizer, FlavaModel

model = FlavaModel.from_pretrained("facebook/flava-full")
tokenizer = BertTokenizer.from_pretrained("facebook/flava-full")

inputs = tokenizer(text=["a photo of a dog"], return_tensors="pt", padding="max_length", max_length=77)

text_embedding = model.get_text_features(**inputs)